In [24]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score
import re
import xgboost, textblob, string
import emoji
import warnings 
warnings.filterwarnings("ignore")
import os

В качестве заготовки для задания прогоним часть 2ого домашнего задания. Нам необходимо получить разреженные матрицы, используя CountVectorizer, TfidfVectorizer для 'tweet_stemmed' и 'tweet_lemmatized' столбцов (4 матрицы).

#### Задание 1.
Построим модель LogisticRegression, используя Bag-of-Words признаки для столбца combine_df['tweet_stemmed']. 
- Поделим Bag-of-Words признаки на train, test (train заканчивается на 31962 строке combine_df)
- Ответами является столбец train_df['label']
- Рассчитаем predict_proba, приведем prediction в в бинарный вид: если предсказание >= 0.3 то 1, иначе 0, тип заменим на int
- Рассчитаем f1_score 

Повторим аналогично для столбца combine_df['tweet_lemmatized'].


In [26]:
#Предобработанные данные взяты с прошлого ДЗ
combine_df = pd.read_pickle('preproc.pkl')
combine_df.head(3)

,id,label,tweet,clean_tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


In [27]:
combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                49159 non-null  int64  
 1   label             31962 non-null  float64
 2   tweet             49159 non-null  object 
 3   clean_tweet       49159 non-null  object 
 4   tweet_token       49159 non-null  object 
 5   tweet_stemmed     49159 non-null  object 
 6   tweet_lemmatized  49159 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.6+ MB


In [28]:
#Уберем пропуски, признаки преобразуем в целочисленные
combine_df = combine_df.dropna()
combine_df.label = combine_df.label.astype(int)
combine_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31962 entries, 0 to 31961
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                31962 non-null  int64 
 1   label             31962 non-null  int32 
 2   tweet             31962 non-null  object
 3   clean_tweet       31962 non-null  object
 4   tweet_token       31962 non-null  object
 5   tweet_stemmed     31962 non-null  object
 6   tweet_lemmatized  31962 non-null  object
dtypes: int32(1), int64(1), object(5)
memory usage: 1.8+ MB


In [29]:
#Для разбивки используем 75Х25
SSIZE = int(len(combine_df) * 0.75)
SSIZE

23971

In [36]:
# stemmed
(X_train_stemmed, y_train_stemmed), (X_test_stemmed, y_test_stemmed) = (combine_df.tweet_stemmed.apply(lambda x: ' '.join(x)).values[:SSIZE], combine_df.label.tolist()[:SSIZE]), (combine_df.tweet_stemmed.apply(lambda x: ' '.join(x)).values[SSIZE:], combine_df.label.tolist()[SSIZE:])

X_train_stemmed.shape, len(y_train_stemmed), X_test_stemmed.shape, len(y_test_stemmed)

((23971,), 23971, (7991,), 7991)

In [37]:
#Ф-ция тренировки модели
#predict_proba, если предсказание >= 0.3 то 1, иначе 0

def train_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)
    return f1_score(y_test, preds[:,1]>0.3)

In [38]:
#Рассчитаем f1_score для CountVectorizer
count_vec = CountVectorizer(max_df=0.9, stop_words='english', max_features=1000)
xtrain_stemmed_count_vec = count_vec.fit_transform(X_train_stemmed)
xtest_stemmed_count_vec = count_vec.transform(X_test_stemmed)

clf = linear_model.LogisticRegression()
f1_score_stemmed_count_vec = train_model(
    clf, 
    xtrain_stemmed_count_vec, 
    y_train_stemmed, 
    xtest_stemmed_count_vec, 
    y_test_stemmed)
f1_score_stemmed_count_vec

0.5579937304075234

In [39]:
#Рассчитаем f1_score для TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=5000)
xtrain_stemmed_tfidf_vec = count_vec.fit_transform(X_train_stemmed)
xtest_stemmed_tfidf_vec = count_vec.transform(X_test_stemmed)

clf = linear_model.LogisticRegression()
f1_score_stemmed_tfidf_vec = train_model(
    clf, 
    xtrain_stemmed_tfidf_vec, 
    y_train_stemmed, 
    xtest_stemmed_tfidf_vec, 
    y_test_stemmed)
f1_score_stemmed_tfidf_vec

0.5579937304075234

In [41]:
# lemmatized
(X_train_lemmatized, y_train_lemmatized), (X_test_lemmatized, y_test_lemmatized) = (
    combine_df.tweet_lemmatized.apply(lambda x: ' '.join(x)).values[:SSIZE], combine_df.label.tolist()[:SSIZE]
), (
    combine_df.tweet_lemmatized.apply(lambda x: ' '.join(x)).values[SSIZE:], combine_df.label.tolist()[SSIZE:]
)

In [42]:
#Рассчитаем f1_score для CountVectorizer

count_vec = CountVectorizer(max_df=0.9, stop_words='english', max_features=1000)

xtrain_lemmatized_count_vec = count_vec.fit_transform(X_train_lemmatized)
xtest_lemmatized_count_vec = count_vec.transform(X_test_lemmatized)

clf = linear_model.LogisticRegression()

f1_score_lemmatized_count_vec = train_model(
    clf, 
    xtrain_lemmatized_count_vec, 
    y_train_lemmatized, 
    xtest_lemmatized_count_vec, 
    y_test_lemmatized)
f1_score_lemmatized_count_vec

0.5473908413205538

In [43]:
#Рассчитаем f1_score для TfidfVectorizer

tfidf_vec = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=1000)
xtrain_lemmatized_tfidf_vec = tfidf_vec.fit_transform(X_train_lemmatized)
xtest_lemmatized_tfidf_vec = tfidf_vec.transform(X_test_lemmatized)

clf = linear_model.LogisticRegression()

f1_score_lemmatized_tfidf_vec = train_model(
    clf, 
    xtrain_lemmatized_tfidf_vec, 
    y_train_lemmatized, 
    xtest_lemmatized_tfidf_vec, 
    y_test_lemmatized)
f1_score_lemmatized_tfidf_vec

0.5497326203208557

#### Задание 2.
Построим модель LogisticRegression, используя TF-IDF признаки для столбца combine_df['tweet_stemmed']. 
- Поделим TF-IDF признаки на train, test (train заканчивается на 31962 строке combine_df)
- Ответами является столбец train_df['label']
- Рассчитаем predict_proba, приведем prediction в в бинарный вид: если предсказание >= 0.3 то 1, иначе 0, тип заменим на int
- Рассчитаем f1_score 

Повторим аналогично для столбца combine_df['tweet_lemmatized'].


#### Задание 3.
Выведите результаты f1-score всех моделей, сделайте вывод.


In [44]:
f1_score_stemmed_count_vec

0.5579937304075234

In [45]:
f1_score_stemmed_tfidf_vec

0.5579937304075234

In [46]:
f1_score_lemmatized_count_vec

0.5473908413205538

In [47]:
f1_score_lemmatized_tfidf_vec

0.5497326203208557

#### Вывод:
Стемминг показал лучший результат. CountVectorizer и TfidfVectorizer показали одинаковый результат

#### Задание 4.
Теперь перейдем к визуализации. Посмотрим, какие слова являются наиболее популярные в датасете с помощью облака слов (WordCloud).
Облако слов - это визуализация, в которой наиболее частые слова большого размера, а менее частые слова меньшего размера.
- объединим слова в одну строку
- создадим словарь частот слов с помощью collections.Counter
- нарисуем облако слов с частотами слов с помощью WordCloud.generate_from_frequencies()
- используем nltk.corpus.stopwords как параметр stopwords, чтобы убрать "мусорные" частотные слова


#### На 3.8 pithon не смогла установить WordCloud без ошибок

#### Задание 5.
Теперь отобразим облако слов для отзывов, не содержащих токсичных комментариев (combine_df['label'] == 0). 


#### Задание 6.
Теперь отобразим облако слов для отзывов, содержащих токсичные комментарии (combine_df['label'] == 1). 
